# Noyau Sémantique

Dans cet exemple de code, vous utiliserez le framework d'IA [Noyau Sémantique](https://aka.ms/ai-agents-beginners/semantic-kernel) pour créer un agent basique.

L'objectif de cet exemple est de vous montrer les étapes que nous utiliserons par la suite dans les exemples de code supplémentaires lors de l'implémentation des différents modèles agentiques.


## Importez les packages Python nécessaires


In [ ]:
import json
import os 

from typing import Annotated

from dotenv import load_dotenv

from IPython.display import display, HTML

from openai import AsyncOpenAI

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent
from semantic_kernel.functions import kernel_function

## Création du client

Dans cet exemple, nous utiliserons [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) pour accéder au LLM.

L'`ai_model_id` est défini comme `gpt-4o-mini`. Essayez de changer le modèle pour un autre disponible sur le marketplace de GitHub Models afin de voir les différents résultats.

Pour utiliser le `Azure Inference SDK`, qui est utilisé pour le `base_url` des GitHub Models, nous utiliserons le connecteur `OpenAIChatCompletion` au sein de Semantic Kernel. Il existe également d'autres [connecteurs disponibles](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion) pour utiliser Semantic Kernel avec d'autres fournisseurs de modèles.


In [ ]:
import random   

# Define a sample plugin for the sample

class DestinationsPlugin:
    """A List of Random Destinations for a vacation."""

    def __init__(self):
        # List of vacation destinations
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        # Track last destination to avoid repeats
        self.last_destination = None

    @kernel_function(description="Provides a random vacation destination.")
    def get_random_destination(self) -> Annotated[str, "Returns a random vacation destination."]:
        # Get available destinations (excluding last one if possible)
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        # Select a random destination
        destination = random.choice(available_destinations)

        # Update the last destination
        self.last_destination = destination

        return destination

In [ ]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## Création de l'Agent

Ci-dessous, nous créons l'Agent appelé `TravelAgent`.

Pour cet exemple, nous utilisons des instructions très simples. Vous pouvez modifier ces instructions pour observer comment l'agent réagit différemment.


In [ ]:
agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin()],
    name="TravelAgent",
    instructions="You are a helpful AI Agent that can help plan vacations for customers at random destinations",
)

## Exécution des Agents

Nous pouvons maintenant exécuter l'Agent en définissant le `ChatHistory` et en y ajoutant le `system_message`. Nous utiliserons les `AGENT_INSTRUCTIONS` que nous avons définies précédemment.

Une fois ces éléments définis, nous créons un `user_inputs`, qui correspond à ce que l'utilisateur envoie à l'agent. Dans ce cas, nous avons défini ce message comme étant `Planifie-moi des vacances ensoleillées`.

N'hésitez pas à modifier ce message pour voir comment l'agent répond différemment.


In [ ]:
user_inputs = [
    "Plan me a day trip.",
    "I don't like that destination. Plan me another vacation.",
]

async def main():
    thread: ChatHistoryAgentThread | None = None

    for user_input in user_inputs:
        html_output = (
            f"<div style='margin-bottom:10px'>"
            f"<div style='font-weight:bold'>User:</div>"
            f"<div style='margin-left:20px'>{user_input}</div></div>"
        )

        agent_name = None
        full_response: list[str] = []
        function_calls: list[str] = []

        # Buffer to reconstruct streaming function call
        current_function_name = None
        argument_buffer = ""

        async for response in agent.invoke_stream(
            messages=user_input,
            thread=thread,
        ):
            thread = response.thread
            agent_name = response.name
            content_items = list(response.items)

            for item in content_items:
                if isinstance(item, FunctionCallContent):
                    if item.function_name:
                        current_function_name = item.function_name

                    # Accumulate arguments (streamed in chunks)
                    if isinstance(item.arguments, str):
                        argument_buffer += item.arguments
                elif isinstance(item, FunctionResultContent):
                    # Finalize any pending function call before showing result
                    if current_function_name:
                        formatted_args = argument_buffer.strip()
                        try:
                            parsed_args = json.loads(formatted_args)
                            formatted_args = json.dumps(parsed_args)
                        except Exception:
                            pass  # leave as raw string

                        function_calls.append(f"Calling function: {current_function_name}({formatted_args})")
                        current_function_name = None
                        argument_buffer = ""

                    function_calls.append(f"\nFunction Result:\n\n{item.result}")
                elif isinstance(item, StreamingTextContent) and item.text:
                    full_response.append(item.text)

        if function_calls:
            html_output += (
                "<div style='margin-bottom:10px'>"
                "<details>"
                "<summary style='cursor:pointer; font-weight:bold; color:#0066cc;'>Function Calls (click to expand)</summary>"
                "<div style='margin:10px; padding:10px; background-color:#f8f8f8; "
                "border:1px solid #ddd; border-radius:4px; white-space:pre-wrap; font-size:14px; color:#333;'>"
                f"{chr(10).join(function_calls)}"
                "</div></details></div>"
            )

        html_output += (
            "<div style='margin-bottom:20px'>"
            f"<div style='font-weight:bold'>{agent_name or 'Assistant'}:</div>"
            f"<div style='margin-left:20px; white-space:pre-wrap'>{''.join(full_response)}</div></div><hr>"
        )

        display(HTML(html_output))

await main()


---

**Avertissement** :  
Ce document a été traduit à l'aide du service de traduction automatique [Co-op Translator](https://github.com/Azure/co-op-translator). Bien que nous nous efforcions d'assurer l'exactitude, veuillez noter que les traductions automatisées peuvent contenir des erreurs ou des inexactitudes. Le document original dans sa langue d'origine doit être considéré comme la source faisant autorité. Pour des informations critiques, il est recommandé de recourir à une traduction professionnelle réalisée par un humain. Nous déclinons toute responsabilité en cas de malentendus ou d'interprétations erronées résultant de l'utilisation de cette traduction.
